# ДЗ 2

In [10]:
from cloud_ml.storage.api import Storage

# To retrieve client secret:
# 1. Go to link: https://developers.google.com/drive/api/v3/quickstart/python
# 2. Press "Enable the drive API"
# 3. Choose "TVs and limited input devices"
client_secret = {8:")"}

# downloading contents of the remote file into the local one
gdrive = Storage.gdrive(client_secret)
gdrive_file_id = '1CUZnBtYwifVXS21yVg62T-vrPVayso5H'
dst_path = 'data/nturgbd_skeletons_s001_to_s017.zip'
gdrive.get(gdrive_file_id, dst_path)

In [11]:
from zipfile import ZipFile

In [12]:
# fname = 'datasets.zip'
zipfile = ZipFile(dst_path)
zipfile.extractall()

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:475: UserWarning: The following variables cannot be serialized: zipfile
  warnings.warn(message)


In [17]:
client_secret = {"installed":{"client_id":"857370663792-fiev084kokkeivk2qbvq95qvc9gcs6h0.apps.googleusercontent.com","project_id":"quickstart-1606583721372","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"DKtGi6FZOWvWrPhr5Irp-Wbm","redirect_uris":["urn:ietf:wg:oauth:2.0:oob","http://localhost"]}}
gdrive = Storage.gdrive(client_secret)

gdrive_file_ms_id = '1eCKZ_4MEGHE3IK0t-tS-AOF3D4lCXkdN'
file_path_sampl_miss = 'NTU_RGBD_samples_with_missing_skeletons.txt'
gdrive.get(gdrive_file_ms_id, file_path_sampl_miss)

## Задание 1

Собрать класс датасет с функциями из ноутбука по генерации датасета. Надо функции перенести в класс + добавить возможность при инициализации менять длину последовательности кадров.

In [118]:
#!L
import os

import numpy as np
import pandas as pd

import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

In [22]:
#!L
class Croped_Raw_Data:
    def __init__(self, data_path, broken_files_path):
        subjects = list(range(0, 28)) #количество людей выполняющих действия
        classes = [8, 10, 22, 23, 27, 21] #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
        cameras = [1, 2, 3] 
        
        labels = []
        counter = 0
        files_counter = {}
        
        self.__files = []
        self.__action_classes = {}

        with open(broken_files_path, 'r') as f:
            broken_files = f.read().split("\n")

        raw_files = os.listdir(data_path)

        for filename in raw_files:
            if filename not in broken_files:
                action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                if action_class in classes and camera_id in cameras:  #and subject_id in subjects:
                    if action_class in self.__action_classes:
                        if files_counter[action_class] < 120:
                            self.__files.append([filename, self.__action_classes[action_class]])
                            files_counter[action_class] = files_counter[action_class] + 1
                    else:
                        self.__action_classes.update({action_class : counter})
                        files_counter.update({action_class : 1})
                        counter += 1
                        self.__files.append([filename, self.__action_classes[action_class]])
                        
    
    def get_files(self):
        return self.__files
    
    
    def get_classes(self):
        return self.__action_classes

In [96]:
#!L
data_path = "nturgb+d_skeletons/"
broken_files_path = "NTU_RGBD_samples_with_missing_skeletons.txt"

data = Croped_Raw_Data(data_path, broken_files_path)

In [97]:
#!L
data.get_classes()

{22: 0, 23: 1, 8: 2, 27: 3, 10: 4, 21: 5}

In [57]:
#!L
a, b = torch.utils.data.random_split(range(10), [3, 7], generator=torch.Generator().manual_seed(42))
print(a)

In [107]:
#!L
class Skeleton_Dataset(Dataset):
    def __init__(self, 
                 inc_data, 
                 data_path, 
                 transform=None, 
                 chonk_len=45):
        
        self.transform = transform
        self.chonk_len = chonk_len
        self.data_path = data_path

        data_lst = []
        labels_lst = []
        numbers = {v: 0 for k, v in inc_data.get_classes().items()}

        for file in inc_data.get_files():
            frames_blocks, label = self.create_coords_blocks(file)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data_lst = data_lst + frames_blocks
                labels_lst = labels_lst + label
        data_np = np.asarray(data_lst)
        labels_np = np.asarray(labels_lst)

        data_sq = data_np.reshape(len(data_np), -1)
        self.data = pd.DataFrame(data_sq)
        self.data['labels'] = labels_np
        self.labels = self.data.iloc[:,-1]


    def __len__(self):
        return len(self.data) 
    
    
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(self.chonk_len, 25*3)
        label = self.labels.values[idx]
        if self.transform != None:
            item = self.transform(item)
        return (item, label)
    
    
    def read_skeleton_filter(self, file):
        with open(file, 'r') as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []

                for m in range(frame_info['numBody']):
                    body_info = {}
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)

        return skeleton_sequence


    def read_xyz(self, file, max_body=1, num_joint=25):
        seq_info = self.read_skeleton_filter(file)
        data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < max_body and j < num_joint:
                        data[m, n, j, :] = [v['x'], v['y'], v['z']]
        return data


    def create_coords_blocks(self, test_file):   
        frame_counter = 0
        new_labels = []
        new_frames = []
        blocks = []

        test_frames = self.read_xyz(self.data_path + test_file[0])[0]
        label = test_file[1]
        slice_len = self.chonk_len * int(len(test_frames)/self.chonk_len)


        for index in range(len(test_frames[:slice_len])):
            frame_counter += 1
            new_frames.append(test_frames[index].flatten())
            if frame_counter == self.chonk_len:
                frame_counter = 0
                blocks.append(np.array(new_frames))
                new_labels = new_labels + [label]
                new_frames = []

        return blocks, new_labels

In [108]:
#!L
dataset = Skeleton_Dataset(data, data_path)

In [109]:
#!L
dataset.data[:int(0.75*len(dataset.data))].shape

(586, 3376)

In [110]:
#!L
label = dataset.labels.values[41]
label

2

In [111]:
#!L
skel, lab = dataset.__getitem__(41)

In [112]:
#!L
lab

2

In [113]:
#!L
skel.shape

(45, 75)

In [115]:
#!L
action_classes = [
    "drink water",
    "eat meal/snack",
    "brushing teeth",
    "brushing hair",
    "drop",
    "pickup",
    "throw",
    "sitting down",
    "standing up (from sitting position)",
    "clapping",
    "reading",
    "writing",
    "tear up paper",
    "wear jacket",
    "take off jacket",
    "wear a shoe",
    "take off a shoe",
    "wear on glasses",
    "take off glasses",
    "put on a hat/cap",
    "take off a hat/cap",
    "cheer up",
    "hand waving",
    "kicking something",
    "reach into pocket",
    "hopping (one foot jumping)",
    "jump up",
    "make a phone call/answer phone",
    "playing with phone/tablet",
    "typing on a keyboard",
    "pointing to something with finger",
    "taking a selfie",
    "check time (from watch)",
    "rub two hands together",
    "nod head/bow",
    "shake head",
    "wipe face",
    "salute",
    "put the palms together",
    "cross hands in front (say stop)",
    "sneeze/cough",
    "staggering",
    "falling",
    "touch head (headache)",
    "touch chest (stomachache/heart pain)",
    "touch back (backache)",
    "touch neck (neckache)",
    "nausea or vomiting condition",
    "use a fan (with hand or paper)/feeling warm",
    "punching/slapping other person",
    "kicking other person",
    "pushing other person",
    "pat on back of other person",
    "point finger at the other person",
    "hugging other person",
    "giving something to other person",
    "touch other person's pocket",
    "handshaking",
    "walking towards each other",
    "walking apart from each other",
    "put on headphone",
    "take off headphone",
    "shoot at the basket",
    "bounce ball",
    "tennis bat swing",
    "juggling table tennis balls",
    "hush (quite)",
    "flick hair",
    "thumb up",
    "thumb down",
    "make ok sign",
    "make victory sign",
    "staple book",
    "counting money",
    "cutting nails",
    "cutting paper (using scissors)",
    "snapping fingers",
    "open bottle",
    "sniff (smell)",
    "squat down",
    "toss a coin",
    "fold paper",
    "ball up paper",
    "play magic cube",
    "apply cream on face",
    "apply cream on hand back",
    "put on bag",
    "take off bag",
    "put something into a bag",
    "take something out of a bag",
    "open a box",
    "move heavy objects",
    "shake fist",
    "throw up cap/hat",
    "hands up (both hands)",
    "cross arms",
    "arm circles",
    "arm swings",
    "running on the spot",
    "butt kicks (kick backward)",
    "cross toe touch",
    "side kick",
    "yawn",
    "stretch oneself",
    "blow nose",
    "hit other person with something",
    "wield knife towards other person",
    "knock over other person (hit with body)",
    "grab other person’s stuff",
    "shoot at other person with a gun",
    "step on foot",
    "high-five",
    "cheers and drink",
    "carry something with other person",
    "take a photo of other person",
    "follow other person",
    "whisper in other person’s ear",
    "exchange things with other person",
    "support somebody with hand",
    "finger-guessing game (playing rock-paper-scissors)"
]
LABELS = {0: "cheer up", 1: "jump up", 2:  "hand waving", 3: "sitting down", 4: "clapping"}
classes = [8, 10, 22, 23, 27, 21]    
LABELS = {i: action_classes[classes[i]] for i in range(6)}
LABELS

{0: 'standing up (from sitting position)',
 1: 'reading',
 2: 'hand waving',
 3: 'kicking something',
 4: 'make a phone call/answer phone',
 5: 'cheer up'}

In [116]:
#!L
LABELS[lab]

'hand waving'

## Задание 2

Построить график зависимости от количества кадров в последовательности (уменьшать и увеличивать)

In [119]:
#!L
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [120]:
#!L
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [121]:
#!L
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [122]:
#!L
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
#     print(output.topk(5))
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [123]:
#!L
split_train_part = 0.75
train_dataset, test_dataset = torch.utils.data.random_split(dataset, 
                                                            [int(split_train_part*dataset.data.shape[0]), 
                                                             dataset.data.shape[0] - int(split_train_part*dataset.data.shape[0])])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=False)

In [124]:
#!L
from torch import optim
import time
import math

criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(), lr=learning_rate, momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(200):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 1.7793  / standing up (from sitting position) ✗ (reading)
epoch : 13 iter : 19 (0m 20s) 1.7650  / hand waving ✗ (make a phone call/answer phone)
epoch : 27 iter : 1 (0m 41s) 1.5809  / standing up (from sitting position) ✗ (make a phone call/answer phone)
epoch : 40 iter : 20 (1m 1s) 1.7983  / reading ✗ (make a phone call/answer phone)
epoch : 54 iter : 2 (1m 22s) 1.3202  / reading ✓
epoch : 67 iter : 21 (1m 42s) 1.2090  / standing up (from sitting position) ✓
epoch : 81 iter : 3 (2m 3s) 0.9431  / kicking something ✓
epoch : 94 iter : 22 (2m 24s) 0.9488  / make a phone call/answer phone ✗ (cheer up)
epoch : 108 iter : 4 (2m 44s) 1.0420  / reading ✗ (cheer up)
epoch : 121 iter : 23 (3m 4s) 1.3536  / make a phone call/answer phone ✓
epoch : 135 iter : 5 (3m 25s) 0.6645  / reading ✓
epoch : 148 iter : 24 (3m 46s) 1.1627  / make a phone call/answer phone ✗ (reading)
epoch : 162 iter : 6 (4m 6s) 0.4204  / cheer up ✗ (reading)
epoch : 175 iter : 25 (4m 27s) 0.6787  

In [126]:
#!L
data_path = "nturgb+d_skeletons/"
broken_files_path = "NTU_RGBD_samples_with_missing_skeletons.txt"

data = Croped_Raw_Data(data_path, broken_files_path)
dataset_c35 = Skeleton_Dataset(data, data_path, chonk_len=35)

In [131]:
#!L
dataset_c35.data.shape

(905, 2626)

In [132]:
#!L
skel, lab = dataset_c35.__getitem__(41)

In [133]:
#!L
skel.shape

(35, 75)

In [134]:
#!L
lab

5

In [135]:
#!L
split_train_part = 0.75
train_dataset, test_dataset = torch.utils.data.random_split(dataset_c35, 
                                                            [int(split_train_part*dataset_c35.data.shape[0]), 
                                                             dataset_c35.data.shape[0] - int(split_train_part*dataset_c35.data.shape[0])])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=False)

In [137]:
#!L
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=6, bias=True)
)

In [138]:
#!L
from torch import optim
import time
import math

criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(), lr=learning_rate, momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(200):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 1.8289  / standing up (from sitting position) ✓
epoch : 11 iter : 27 (0m 17s) 1.7605  / hand waving ✗ (cheer up)
epoch : 23 iter : 11 (0m 34s) 1.5320  / hand waving ✗ (cheer up)
epoch : 34 iter : 38 (0m 51s) 1.4354  / standing up (from sitting position) ✗ (cheer up)
epoch : 46 iter : 22 (1m 8s) 1.6923  / hand waving ✗ (cheer up)
epoch : 58 iter : 6 (1m 25s) 1.2300  / hand waving ✗ (kicking something)
epoch : 69 iter : 33 (1m 42s) 1.1184  / kicking something ✓
epoch : 81 iter : 17 (1m 59s) 1.0343  / hand waving ✗ (kicking something)
epoch : 93 iter : 1 (2m 16s) 1.1912  / kicking something ✗ (hand waving)
epoch : 104 iter : 28 (2m 33s) 0.8668  / reading ✓
epoch : 116 iter : 12 (2m 50s) 0.7947  / kicking something ✗ (hand waving)
epoch : 127 iter : 39 (3m 7s) 1.0142  / hand waving ✗ (make a phone call/answer phone)
epoch : 139 iter : 23 (3m 24s) 1.1213  / reading ✗ (standing up (from sitting position))
epoch : 151 iter : 7 (3m 41s) 0.8545  / make a phone call/an

## Задание 3

Построить график зависимости от количества модулей LSTM

In [ ]:
#!L
